# Project 01 - Color Compression

## Thông tin sinh viên

- Họ và tên: Lê Hồng Ngọc
- MSSV: 23127236
- Lớp: 23CLC05

## Import các thư viện liên quan

In [55]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Helper functions

In [ ]:
def read_img(img_path):
    '''
    Read image from img_path

    Parameters
    ----------
    img_path : str
        Path of image

    Returns
    -------
        Image (2D)
    '''

    # YOUR CODE HERE
    img = Image.open(img_path)
    
    if img.mode != 'RGB':
        img = img.convert('RGB')

    img_array = np.array(img)
    img.close()

    return img_array


def show_img(img_2d):
    '''
    Show image

    Parameters
    ----------
    img_2d : <your type>
        Image (2D)
    '''

    # YOUR CODE HERE
    plt.imshow(img_2d)
    plt.axis('off')
    plt.show()


def save_img(img_2d, img_path):
    '''
    Save image to img_path

    Parameters
    ----------
    img_2d : <your type>
        Image (2D)
    img_path : str
        Path of image
    '''

    # YOUR CODE HERE

    img_2d = np.array(img_2d).astype(np.uint8)
    plt.imsave(img_path, img_2d, vmin = 0, vmax = 255)


def convert_img_to_1d(img_2d):
    '''
    Convert 2D image to 1D image

    Parameters
    ----------
    img_2d : <your type>
        Image (2D)

    Returns
    -------
        Image (1D)
    '''

    # YOUR CODE HERE
    height = img_2d.shape[0]  
    width = img_2d.shape[1]  

    return img_2d.reshape(height * width, 3)


def kmeans(img_1d, k_clusters, max_iter, init_centroids='random'):
    '''
    K-Means algorithm

    Parameters
    ----------
    img_1d : np.ndarray with shape=(height * width, num_channels)
        Original (1D) image
    k_clusters : int
        Number of clusters
    max_iter : int
        Max iterator
    init_centroids : str, default='random'
        The method used to initialize the centroids for K-means clustering
        'random' --> Centroids are initialized with random values between 0 and 255 for each channel
        'in_pixels' --> A random pixel from the original image is selected as a centroid for each cluster

    Returns
    -------
    centroids : np.ndarray with shape=(k_clusters, num_channels)
        Stores the color centroids for each cluster
    labels : np.ndarray with shape=(height * width, )
        Stores the cluster label for each pixel in the image
    '''
    
    # YOUR CODE HERE
    num_pixels = img_1d.shape[0]
    num_channels = img_1d.shape[1]

    if init_centroids == 'random':
        centroids = np.random.randint(0, 256, size=(k_clusters, num_channels))
    elif init_centroids == 'in_pixels':
        random_indices = np.random.choice(num_pixels, k_clusters, replace=False)
        centroids = img_1d[random_indices].copy()
    else:
        raise ValueError("Invalid initialization method. Use 'random' or 'in_pixels'.")
    
    for _ in range(max_iter):
        dis_square =np.sum(np.square(img_1d.reshape(-1, 1, 3) - centroids.reshape(1, -1, 3)), axis=2)
        labels = np.argmin(dis_square, axis=1)

        new_centroids = np.array([img_1d[labels == k].mean(axis=0) if np.any(labels == k) else centroids[k]
                                for k in range(k_clusters)])
        
        if np.allclose(centroids, new_centroids):
            break
            
        centroids = new_centroids
    
    return centroids, labels


def generate_2d_img(img_2d_shape, centroids, labels):
    '''
    Generate a 2D image based on K-means cluster centroids

    Parameters
    ----------
    img_2d_shape : tuple (height, width, 3)
        Shape of image
    centroids : np.ndarray with shape=(k_clusters, num_channels)
        Store color centroids
    labels : np.ndarray with shape=(height * width, )
        Store label for pixels (cluster's index on which the pixel belongs)

    Returns
    -------
        New image (2D)
    '''

    # YOUR CODE HERE
    centroids = np.array(centroids)
    new_img = centroids[labels].reshape(img_2d_shape).astype(np.uint8)
    return new_img


# Your additional functions here
def count_unique_colors(img_2d):
    unique_colors = len(np.unique(convert_img_to_1d(img_2d), axis=0))
    return unique_colors


## Your tests

In [76]:
# YOUR CODE HERE
def test_functions():
    '''
    Test all helper functions
    '''

    test_img_path = r"C:\Users\LENOVO\Downloads\wilamowice-5249675_1280.jpg"
    img = read_img(test_img_path)
    img_1d = convert_img_to_1d(img)
    centroids, labels = kmeans(img_1d, 50, max_iter=100, init_centroids='in_pixels')
    new_img = generate_2d_img(img.shape, centroids, labels)
    show_img(img)
    show_img(new_img)
    save_img(new_img, "50_pixel.png")
    print("All tests completed successfully")

## Main FUNCTION

In [ ]:
# YOUR CODE HERE
def main():
    '''
    Main function to run the color reduction program
    '''
    print("=== Image Color Reduction using K-Means ===")

    img_path = input("Enter image file path: ").strip()
    
    while True:
        k = int(input("Enter number of colors to reduce to (2-256): "))
        if 2 <= k <= 256:
            break
        print("Please enter a number between 2 and 256")
    
    img = read_img(img_path)
    color_count = count_unique_colors(img)
    img_1d = convert_img_to_1d(img)


    print(f"Running K-Means with {k} clusters...")
    centroids, labels = kmeans(img_1d, k, max_iter=100, init_centroids='random')
    
    new_img = generate_2d_img(img.shape, centroids, labels)
    
    print(f"\nOriginal Image ({color_count} colors):")
    show_img(img)
    
    print(f"\nReduced Image ({k} colors):")
    show_img(new_img)
    
    output_path = input(f"\nEnter output file name: ").strip() # Ensure the output path has a valid image extension e.g., .png or .pdf
    save_img(new_img, output_path)

In [ ]:
# Call main function
if __name__ == "__main__":
    main()